In [1]:
// load libraries and apply configuration
// in q mem space


// in python mem space



// data loading
weather:.Q.id("  II ***",(24*2)#"FS";enlist ";")0:`$":./abr_meteo23.csv";
traffic:.Q.id("IPS  J S";enlist ";")0:`$":./04-2023.csv";
weather_station:.Q.id("SISS";enlist ";")0:`$":./Estaciones_control_datos_meteorologicos.csv";
traffic_station:.Q.id("SISSSSSSS";enlist ";")0:`$":./pmed_ubicacion_04-2023.csv";

// preprocess weather data
weather:`ESTACION`FECHA xcols `ANO`MES`DIA _update FECHA:"D"$(raze')flip(ANO;MES;DIA) from weather;
cw:cols weather;
mc:(neg 2*24)_ cw;
tr:{(flip;enlist,cw where cw like x,"*")};
ops:(mc,`HORA`VALOR`OK)!mc,((#;(count,`i);(enlist;(*;01:00;til,24)));tr"H";tr"V");
weather:ungroup?[weather;();0b;ops];
weather:delete from weather where OK<>`V;
weather:update FECHA:("P"$((string FECHA),'" ",'string"t"$HORA))from weather;
weather:update ESTACION:"j"$ESTACION from weather;
weather:delete VALIDO,HORA from weather;
mag:80 81 82 83 86 87 88 89!`ULTRAVIOLETA`VIENTO`DIRECCION`TEMPERATURA`HUMEDAD`PRESION`SOLAR`PRECIPITACION;
weather:update mag MAGNITUD from weather;
u:value mag;
weather:0!exec u#MAGNITUD!VALOR by FECHA,ESTACION from weather;
weather:lower[cols weather]xcol weather;

// preprocess traffic data
traffic:`fecha xasc select avg carga by fecha,id from traffic where error=`N;

// use PyKX to run cdist function
b:select "F"$string LONGITUD, "F"$string LATITUD from weather_station;
a:select "F"$string longitud, "F"$string latitud from traffic_station;

distance:{[x1;x2;y1;y2]abs(x1-y1)+abs(x2-y2)};
distance_matrix:distance[b.LONGITUD; b.LATITUD]'[a.longitud; a.latitud];


station_index:distance_matrix?'min each distance_matrix;
distance_table:select id,estacion:station_index,"F"$string longitud,"F"$string latitud from traffic_station;

tr:update "I"$string id from 0!traffic;
td:update "I"$string id from distance_table;
uni:tr lj 1!td;
complete:uni lj 2!`fecha`estacion xcols weather;

// prepare table for model
complete:update hour:`hh$fecha,weekday:("d"$fecha)mod 7 from complete;
complete:3!`hour`weekday`estacion xcols 0^complete;
cargas:select carga_avg:avg carga by hour,weekday,estacion from complete;
temps:select temps_avg:avg temperatura by hour from complete;

complete:1!0!complete lj cargas;
complete:update carga_v:carga-carga_avg,temperatura_v:temperatura-temps_avg from 0!complete lj temps;



In [32]:
final: select hour, weekday, fecha, id, carga, temperatura, precipitacion from complete
final: `hour`weekday`date`traffic_station`traffic_load`temperature`rainfall xcol final

final:select from final where weekday>1,9<hour,hour<20
final:select from final where 40 <= (avg; traffic_load) fby traffic_station
final:update traffic_load:traffic_load%100 from final

In [33]:
final

hour weekday date                          traffic_station traffic_load temperature rainfall
--------------------------------------------------------------------------------------------
10   2       2023.04.03D10:00:00.000000000 3488            0.3386523    9.932495    0       
10   2       2023.04.03D10:00:00.000000000 3490            0.4886523    9.932495    0       
10   2       2023.04.03D10:00:00.000000000 3491            0.3124771    -1.467505   0       
10   2       2023.04.03D10:00:00.000000000 3492            0.4224771    -1.467505   0       
10   2       2023.04.03D10:00:00.000000000 3521            0.4373596    8.232495    0       
10   2       2023.04.03D10:00:00.000000000 3532            0.2684373    7.832495    0       
10   2       2023.04.03D10:00:00.000000000 3533            0.2673596    8.232495    0       
10   2       2023.04.03D10:00:00.000000000 3611            0.4086523    9.932495    0       
10   2       2023.04.03D10:00:00.000000000 3705            0.4159759  

In [116]:
prepareData:{[tt;data; test_per_series]
    op:$[tt=`train;#;_];                                      / `train or `test decide the operator
    /m:`rainfall`temperature`traffic_load`hour`weekday;        / the 5 columns we need
    /train_data:?[data;();`traffic_station;m!({((y;test_per_series);x)}[;#]')m]; 
    train_data: select x: (-1*test_per_series)_flip (rainfall;temperature;traffic_load;hour;weekday)  by traffic_station from data; 
    test_data:  select x: (test_per_series)_flip (rainfall;temperature;traffic_load;hour;weekday) by traffic_station from data;
    
    (test_data; train_data)
    /test_data: ?[data;();`traffic_station;m!({((y;test_per_series);x)}[;_]')m];  / functional form. essentially we either pick the first 80 elems or from 80 until the end (# or _)
    /sw:{({y#z _x}[x;5;]')til count b:10_x};                   / sliding window function. takes turbomatrix and divides into chunks of 5x5
    /gl:{6_(flip x)[2]};                                       / gets the load (y data)
    /toMatrix:{({[t;i]value t[i]}[x;]')til count x:flip x};    / table to matrix
    /data:(toMatrix')data;                                     / convert each subtable (data is a keyed table) to a matrix
    /X:(sw')data;                                              / apply sliding window to get X
    /y:(gl')data;                                              / and gl to get y
    /(X;y)                                                     / return both
    }
train: prepareData [`train; final; 80][1]

In [117]:
count (value train[3488])[0]

695
